In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup as bs

In [3]:
import requests

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
#use because using beautifulsoup images weren't retreiving due to lazy loading in website so to wait for site to complete load
# selenium's webdriver is used

In [5]:
url = "https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament"

In [6]:
page = requests.get(url)

In [7]:
soup = bs(page.text, 'html')

In [8]:
# getting match results table (all rows)
match_results = soup.table.tbody.find_all('tr')

In [9]:
cols = ['name', 'team', 'image', 'batting style', 'bowlling style', 'playing role', 'description']
df = pd.DataFrame(columns = cols)
df

,name,team,image,batting style,bowlling style,playing role,description


In [32]:
def profile_fetcher (link):
    # setting up web driver to fetch images 
    options = Options()
    # used so that chrome window won't appear again and again
    options.add_argument('--headless=new')
    driver = webdriver.Chrome(options=options)
    driver.get(link)
    # this will wait for 10 sec (assumed to time for website loading) then it 
    # fetches the web page 
    driver.implicitly_wait(10)
    html_content = driver.page_source
    
    link_soup = bs(html_content, 'html.parser')

    # will be used fetch name , team name and image
    black_box = link_soup.find('div', class_= 'ds-bg-raw-black/85')

    # fetches player profile then from it player style
    player_profile = link_soup.find('div', class_= 'ds-grid').find_all('span')
    player_style = [ style.text.strip() for style in player_profile[3:6]]
    style1 = style2 = style3 = None

    # most players are catagarised in 3 styles but there may some which have 2 aur may less so these
    # condition used
    if len(player_style) >= 1:
        style1 = player_style[0]
    if len(player_style) >= 2:
        style2 = player_style[1]
    if len(player_style) == 3:
        style3 = player_style[2]

    player_style = [style1, style2, style3]
        
    # player description
    player_descrip = link_soup.find('div', class_= 'ci-player-bio-content')
    
    
    name = black_box.h1.text
    team = black_box.span.text
    img = None

    # players may not have image so...
    if black_box.find('img'):
        img = black_box.find('img').get('src')
    description = None
    
    if player_descrip:
         description = player_descrip.text.strip()
        
    row = [name, team, img] + player_style + [description]
    length = len(df)
    df.loc[length] = row
    print("done")
    

In [11]:
# fucntion names are used from previous code that why have wierd name
# it fetches link of player profiles from batting summary
def batting_summary (table ) :
    table = table.tbody.find_all('tr')
    for row in table[:-4] :
        row = row.find('td')
        if len(row.text.strip()) > 1 :
            link = 'https://www.espncricinfo.com' + row.a.get('href')
            profile_fetcher(link)
                   

In [12]:
# fucntion names are used from previous code that why have wierd name
# it fetches link of player profiles from balling summary
def balling_summary (table) :
    table = table.tbody.find_all('tr')
    # traverse each row 
    for row in table :
        # from each row fetch all cells
        row = row.find('td')
        if 1 <= len(row.text.strip()) <= 30 :
            link = 'https://www.espncricinfo.com' + row.a.get('href')
            profile_fetcher(link)


In [13]:
def crawler ( link ) :
    # getting resoponse from match summary site
    link_response = requests.get(link)
    link_soup = bs(link_response.text, 'html')

    # batting table for team 1
    table1 = link_soup.find_all('table')[0]
    # balling table for team 1
    table2 = link_soup.find_all('table')[1]
    # batting table for team 2
    table3 = link_soup.find_all('table')[2]
    # batting table for team 2
    table4 = link_soup.find_all('table')[3]

    # pass table1 and table3 to fetch betting details
    batting_summary (table1)
    batting_summary (table3)

    # pass tab2, tab4 to fetch balling details/summary
    balling_summary (table2)
    balling_summary (table4)

In [14]:
# this list will stores all necessary links which i'm going to visit
# storing links in this helped run program in smallers steps
# so if covered previous steps and stored it then if program fail i don't need to run 
# again previous links i can continue from where i got error
Match_list = []

In [16]:
# Trversseing each row of match results and taking last cell from each row

for row in match_results:
    # fetch last cell from each row
    row_data = row.find_all('td')[-1]
    # fetch link from that cell 
    link = 'https://www.espncricinfo.com' + row_data.find('a').get('href')
    # pass fetched link to crawler 
    Match_list.append(link)


In [31]:
for match in Match_list:
    print(match)

https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/england-vs-pakistan-final-1298179/full-scorecard
https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/england-vs-india-2nd-semi-final-1298178/full-scorecard
https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/new-zealand-vs-pakistan-1st-semi-final-1298177/full-scorecard
https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/india-vs-zimbabwe-42nd-match-group-2-1298176/full-scorecard
https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/bangladesh-vs-pakistan-41st-match-group-2-1298175/full-scorecard
https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/netherlands-vs-south-africa-40th-match-group-2-1298174/full-scorecard
https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/england-vs-sri-lanka-39th-match-group-1-1298173/full-scorecard
https://www.espncricinfo.com/series/icc-men-

In [37]:
uncomment this to run program again

# index = 1
# for match in Match_list[0:10]:
#     print(str(index)+ " " + match)
#     crawler(match)
#     index += 1
# print ("break \n" + '\n')
# for match in Match_list[10:20]:
#     print(str(index)+ " " +match)
#     crawler(match)
#     index += 1
# print ("break \n" + '\n')
# for match in Match_list[20:30]:
#     print(str(index)+ " " +match)
#     crawler(match)
#     index += 1
# print ("break \n" + '\n')
# for match in Match_list[40:]:
#     print(str(index)+ " " +match)
#     crawler(match)
#     index += 1
# print ("break \n" + '\n')

In [52]:
df 

,name,team,image,batting style,bowlling style,playing role,description
0,Mohammad Rizwan,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Wicketkeeper,Wicketkeeper Batter,"For several years, it appeared Mohammad Rizwan..."
1,Babar Azam,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Right arm Offbreak,Batter,"A right-hand, top-order batsman known for his ..."
2,Mohammad Haris,Pakistan,None,Right hand Bat,Right arm Offbreak,Occasional Wicketkeeper,None
3,Shan Masood,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Shaan Masood,Shaani,Left hand Bat,A solid and technically sound left-hand opener...
4,Iftikhar Ahmed,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Right arm Offbreak,Middle order Batter,None
...,...,...,...,...,...,...,...
1086,Divan la Cock,Namibia,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Legbreak,Opening Batter,None
1088,Jan Nicol Loftie-Eaton,Namibia,"https://img1.hscicdn.com/image/upload/f_auto,t...",Left hand Bat,"Right arm Medium, Legbreak",Batter,None
1176,Zawar Farid,|,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Right arm Medium,Bowler,None
1178,Chundangapoyil Rizwan,U.A.E.,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Legbreak Googly,Middle order Batter,None


In [57]:
df = df.reset_index(drop = True)

In [58]:
df.to_csv(r"C:\Users\praja\Downloads\shooting-range-master\Projects\T20_2022_players_analysis\web_scrapping/player_profiles.csv")

In [59]:
df

,name,team,image,batting style,bowlling style,playing role,description
0,Mohammad Rizwan,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Wicketkeeper,Wicketkeeper Batter,"For several years, it appeared Mohammad Rizwan..."
1,Babar Azam,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Right arm Offbreak,Batter,"A right-hand, top-order batsman known for his ..."
2,Mohammad Haris,Pakistan,None,Right hand Bat,Right arm Offbreak,Occasional Wicketkeeper,None
3,Shan Masood,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Shaan Masood,Shaani,Left hand Bat,A solid and technically sound left-hand opener...
4,Iftikhar Ahmed,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Right arm Offbreak,Middle order Batter,None
...,...,...,...,...,...,...,...
209,Divan la Cock,Namibia,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Legbreak,Opening Batter,None
210,Jan Nicol Loftie-Eaton,Namibia,"https://img1.hscicdn.com/image/upload/f_auto,t...",Left hand Bat,"Right arm Medium, Legbreak",Batter,None
211,Zawar Farid,|,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Right arm Medium,Bowler,None
212,Chundangapoyil Rizwan,U.A.E.,"https://img1.hscicdn.com/image/upload/f_auto,t...",Right hand Bat,Legbreak Googly,Middle order Batter,None
